# MAKE SURE YOU ARE RUNNING THE GPU RUNTIME.

You can change to the GPU runtime by Runtime > Change Runtime Type > T4 GPU.

Confirm you have a GPU by running the below code.

In [ ]:
!nvidia-smi

Setup environment

In [ ]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python
!pip install langchain chromadb langchainhub
!pip install --upgrade --quiet  sentence_transformers

Download model. For this example, we are using Mistral 7B Instruct.

In [ ]:
!wget https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q6_K.gguf

# If you already have the model downloaded, or want to save it run the following sections.

Enable Drive Connection by Clicking the Button at Files at tab.

Save Model to Drive

In [ ]:
!cp ./mistral-7b-instruct-v0.1.Q6_K.gguf "/content/drive/MyDrive/LLAMA Models/"

Download Model from Drive

In [ ]:
!cp "/content/drive/MyDrive/LLAMA Models/mistral-7b-instruct-v0.1.Q6_K.gguf" ./mistral-7b-instruct-v0.1.Q6_K.gguf

# Prepare the document. For this example, we are using the LibGuide https://guides.nyu.edu/data

Check here for more about this workflow: https://python.langchain.com/docs/use_cases/question_answering/

In [ ]:
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = WebBaseLoader([
    'https://guides.nyu.edu/data/home',
    'https://guides.nyu.edu/data/llm',
    'https://guides.nyu.edu/data/llm-overview',
    'https://guides.nyu.edu/data/llm-bias',
    'https://guides.nyu.edu/data/llm-research-creative-use',
    'https://guides.nyu.edu/data/chatgpt',
    'https://guides.nyu.edu/data/chatgpt-research',
    'https://guides.nyu.edu/data/chatgpt-developer',
    'https://guides.nyu.edu/data/chatgpt-visual-design',
    'https://guides.nyu.edu/data/chatgpt-campus-life',
    'https://guides.nyu.edu/data/emerging-ai',
    'https://guides.nyu.edu/data/chatbots',
    'https://guides.nyu.edu/data/ai-image-generation',
    'https://guides.nyu.edu/data/ai-local',
    'https://guides.nyu.edu/data/ai-teaching-and-learning',
    'https://guides.nyu.edu/data/ai-citations',
    'https://guides.nyu.edu/data/generative-ai',
    'https://guides.nyu.edu/data/ai-governance',
    'https://guides.nyu.edu/data/ai-business',
    'https://guides.nyu.edu/data/css'
])

text_splitter = RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=0)
splits = text_splitter.split_documents(loader.load())

Initialize embedding LLM, it will download the model



In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings


llm_embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

## Create embeddings

In [ ]:
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents=splits, embedding=llm_embedding, persist_directory="./libguide")
retriever = vectorstore.as_retriever()

# Check VectorStore

In [ ]:
retriever.get_relevant_documents("ChatGPT")

Prepare LLM

In [ ]:
temperature = 0.1
top_k = 40
top_p = 0.9
max_tokens = 2048

In [ ]:
from langchain.llms import LlamaCpp
llm = LlamaCpp(model_path="./mistral-7b-instruct-v0.1.Q6_K.gguf", n_ctx=32768, use_mlock=True, n_gpu_layers=33, temperature=temperature, top_k=top_k, top_p=top_p, max_tokens=max_tokens, streaming=True)

Prepare RAG Chain

In [ ]:
from langchain import hub

rag_prompt = hub.pull("rlm/rag-prompt")

In [ ]:
from langchain.schema.runnable import RunnablePassthrough

rag_chain = {"context": retriever, "question": RunnablePassthrough()} | rag_prompt | llm

In [ ]:
import logging
logging.basicConfig(level=logging.INFO)

def infer_rag(text):

    logging.info("Creating answer for the question:" + text)

    # response = rag_chain.invoke(text)

    # logging.info(response)

    # return response

    response_iterator = rag_chain.stream(text)

    return response_iterator

In [ ]:
import logging
logging.basicConfig(level=logging.INFO)

def infer_llm(text):

    logging.info("Trying to answer the question:" + text)

    # response = llm(prompt=text)

    # # response_content = response["choices"][0]["text"]

    # logging.info(response)

    # return response

    response_iterator = llm.stream(text)

    return response_iterator

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import time

# Text widget for input
input_text = widgets.Textarea(
    value='',
    placeholder='Question',
    description='Input:',
    disabled=False,
    layout={'height': '100px', 'width': '600px'}
)

# Button to execute function
execute_button = widgets.Button(description="Generate")

progress_bar = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Progress:',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    orientation='horizontal'
)


# Textarea widgets to display the results
output1 = widgets.Textarea(
    value='',
    placeholder='Answer with RAG will be displayed here',
    description='Answer with RAG:',
    disabled=True,
    layout={'height': '200px', 'width': '600px'}  # Adjust height and width as needed
)

output2 = widgets.Textarea(
    value='',
    placeholder='Answer without RAG will be displayed here',
    description='Answer without RAG:',
    disabled=True,
    layout={'height': '200px', 'width': '600px'}  # Adjust height and width as needed
)

# Function to be executed
def execute_function(change):
    progress_bar.value = 0
    progress_bar.max = 3  # Set this to a high number for a smooth progress bar
    progress_bar.style.bar_color = 'blue'
    display(progress_bar)

    # Step 1: You might want to update progress bar here
    progress_bar.value += 1

    # output_text_rag = infer_rag(input_text.value)

    # output1.value = output_text_rag

    for completion in infer_rag(input_text.value):
      output1.value += completion

    # Step 2: Another update to the progress bar
    progress_bar.value += 1

    # output_text_llm = infer_llm(input_text.value)

    # output2.value = output_text_llm

    for completion in infer_llm(input_text.value):
      output2.value += completion

    # Step 3: Final update to the progress bar
    progress_bar.value += 1
    progress_bar.bar_style = 'success'

# Attach the function to the button
execute_button.on_click(execute_function)

# Display widgets
display(input_text, execute_button, output1, output2)


For example

What is LLM?